In [1]:
import math

In [2]:
with open('day14.input') as fp:
    puzzle_lines = fp.read().split('\n')
puzzle_lines = puzzle_lines[:-1]
puzzle_lines[-1]

'1 VRVDQ => 2 FXGW'

## Part 1 ##

In [3]:
test0_lines = '''10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL'''.split('\n')
test1_lines = '''9 ORE => 2 A
8 ORE => 3 B
7 ORE => 5 C
3 A, 4 B => 1 AB
5 B, 7 C => 1 BC
4 C, 1 A => 1 CA
2 AB, 3 BC, 4 CA => 1 FUEL'''.split('\n')
test2_lines = '''157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT'''.split('\n')
test3_lines = '''2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF'''.split('\n')
test4_lines = '''171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX'''.split('\n')
tests = (test0_lines, test1_lines, test2_lines, test3_lines, test4_lines)
test_results = (31, 165, 13312, 180697, 2210736)

In [4]:
def stoich(reactions):
    s = {}
    for reaction in reactions:
        reactants, products = reaction.split('=>')
        pterms = products.strip().split()
        prod = pterms[1]
        num = int(pterms[0])
        if prod in s:
            raise ValueError(f'Already seen {prod} as a product')
        s[prod] = {'num': num}
        rterms = reactants.strip().split(',')
        r = {}
        for term in rterms:
            n, i = term.split()
            n = int(n)
            i = i.strip()
            r[i] = n
        s[prod]['reactants'] = r
    return s

In [5]:
def rxns_top_level(rxns):
    rxn_products = set(rxns.keys())
    all_reactants = []
    for prod in rxns:
        all_reactants.extend(rxns[prod]['reactants'].keys())
    top_level = rxn_products - set(all_reactants)
    return top_level
    
def replace(comps, rxns):
    top_level = rxns_top_level(rxns)
    replacement = top_level.intersection(comps.keys())
    while replacement:
        c = replacement.pop()
        nc = comps[c]
        rxn_num, reactants = rxns[c]['num'], rxns[c]['reactants']
        num_rx = math.ceil(nc/rxn_num)
        for r, v in reactants.items():
            if r in comps:
                comps[r] += v*num_rx
            else:
                comps[r] = v*num_rx
        del rxns[c]
        del comps[c]
        top_level = rxns_top_level(rxns)
        replacement = top_level.intersection(comps.keys())
    return comps   

In [6]:
rxns = stoich(test0_lines)
comps = {'FUEL': 1}
comps = replace(comps, rxns)
print(comps)

{'ORE': 31}


In [7]:
for i, test in enumerate(tests):
    rxns = stoich(test)
    comps = {'FUEL': 1}
    comps = replace(comps, rxns)
    print(i, comps['ORE'] == test_results[i])

0 True
1 True
2 True
3 True
4 True


In [8]:
rxns = stoich(puzzle_lines)
comps= {'FUEL': 1}
comps = replace(comps, rxns)
print(comps)

{'ORE': 1185296}


In [9]:
1000000000000//13312

75120192

## Part 2 ##

In [11]:
from fractions import Fraction

In [16]:
def stoich2(reactions):
    s = {}
    for reaction in reactions:
        reactants, products = reaction.split('=>')
        pterms = products.strip().split()
        prod = pterms[1]
        num = int(pterms[0])
        if prod in s:
            raise ValueError(f'Already seen {prod} as a product')
        s[prod] = {'num': num}
        rterms = reactants.strip().split(',')
        r = {}
        for term in rterms:
            n, i = term.split()
            n = int(n)
            i = i.strip()
            r[i] = Fraction(n, num)
        s[prod]['reactants'] = r
        s[prod]['num'] = 1
    return s

In [17]:
rxns = stoich2(test0_lines)
rxns

{'A': {'num': 1, 'reactants': {'ORE': Fraction(1, 1)}},
 'B': {'num': 1, 'reactants': {'ORE': Fraction(1, 1)}},
 'C': {'num': 1, 'reactants': {'A': Fraction(7, 1), 'B': Fraction(1, 1)}},
 'D': {'num': 1, 'reactants': {'A': Fraction(7, 1), 'C': Fraction(1, 1)}},
 'E': {'num': 1, 'reactants': {'A': Fraction(7, 1), 'D': Fraction(1, 1)}},
 'FUEL': {'num': 1, 'reactants': {'A': Fraction(7, 1), 'E': Fraction(1, 1)}}}

In [31]:
def replace2(comps, rxns):
    top_level = rxns_top_level(rxns)
    replacement = top_level.intersection(comps.keys())
    while replacement:
        c = replacement.pop()
        nc = Fraction(comps[c])
        rxn_num, reactants = Fraction(rxns[c]['num']), rxns[c]['reactants']
        num_rx = nc/rxn_num
        for r, v in reactants.items():
            if r in comps:
                comps[r] += v*num_rx
            else:
                comps[r] = v*num_rx
        del rxns[c]
        del comps[c]
        top_level = rxns_top_level(rxns)
        replacement = top_level.intersection(comps.keys())
    return comps   

In [32]:
rxns = stoich2(test2_lines)
comps= {'FUEL': 1}
comps = replace2(comps, rxns)
82892753 == 1000000000000//comps['ORE']

True

In [33]:
rxns = stoich2(test3_lines)
comps= {'FUEL': 1}
comps = replace2(comps, rxns)
5586022  == 1000000000000//comps['ORE']

True

In [34]:
rxns = stoich2(test4_lines)
comps= {'FUEL': 1}
comps = replace2(comps, rxns)
460664   == 1000000000000//comps['ORE']

True

In [38]:
rxns = stoich2(puzzle_lines)
comps = {'FUEL': 1}
comps = replace2(comps, rxns)
print(comps)
print(int(1000000000000//comps['ORE']))

{'ORE': Fraction(22142387454504941, 30481920000)}
1376632


726410.5231725869

In [40]:
rxns = stoich(puzzle_lines)
comps= {'FUEL': 1376631}
comps = replace(comps, rxns)
print(comps)

{'ORE': 999999649614}
